<a href="https://colab.research.google.com/github/Michwynn/Capstone-Team-Air-/blob/Data-Cleaning/Geographical_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import folium
from IPython.display import display
import warnings # supress warnings
warnings.filterwarnings('ignore')
import random
RANDOM_SEED = 44

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Capstone

pd.set_option('display.max_columns', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Capstone


**read dataset**

In [45]:
df = pd.read_csv('assets/cleaned_assets/df_cleaned.csv', index_col = 0)
df.head(3)

,Borough,total_crimes,Population per square kilometre,Property price,id,neighbourhood_cleansed,latitude,longitude,host_since,host_is_superhost,host_has_profile_pic,host_identity_verified,amenities,beds,property_type,room_type,accommodates,minimum_nights,maximum_nights,number_of_reviews,price,amenities_count,closest_station_dist,within_1k_station,closest_POI_dist,within_5k_POI,dist_from_center
0,Barking and Dagenham,38988.0,6214.914626,335683.0,198258,Barking and Dagenham,51.53430,0.08178,2011-08-14,t,t,t,"[""Shared patio or balcony"", ""Elevator"", ""Smoke...",1.0,Private room in rental unit,Private room,1,2,180,41,69.0,32,0.591797,1,7.895814,0,14.796863
1,Barking and Dagenham,38988.0,6214.914626,335683.0,306405,Barking and Dagenham,51.54072,0.15246,2012-01-06,f,t,f,"[""Dryer"", ""TV with standard cable"", ""Long term...",1.0,Private room in rental unit,Private room,1,4,365,3,41.0,6,0.399661,2,12.688768,0,19.730923
2,Barking and Dagenham,38988.0,6214.914626,335683.0,2398015,Barking and Dagenham,51.52610,0.11898,2014-01-12,t,t,t,"[""Cleaning products"", ""Freezer"", ""Shower gel"",...",2.0,Private room in home,Private room,2,3,1125,39,42.0,47,1.673583,0,9.872346,0,17.191080


In [46]:
random.seed(RANDOM_SEED)

# get count for each neighbourhood in a df
neighbourhood_count_df = df.groupby('neighbourhood_cleansed')['id'].count().reset_index()
# no of neighbourhoods
n = len(neighbourhood_count_df)
# generate list of random colour for each neighbourhood
get_colors = lambda n: ["#%06x" % random.randint(0, 0xFFFFFF) for _ in range(n)]
neighbourhood_count_df['colour'] = get_colors(n)
# create dict for colour
neighbourhood_colour_dict = pd.Series(neighbourhood_count_df.colour.values,index=neighbourhood_count_df.neighbourhood_cleansed).to_dict()
neighbourhood_count_df.head(3)

,neighbourhood_cleansed,id,colour
0,Barking and Dagenham,435,#d12b9d
1,Barnet,1648,#3bba42
2,Bexley,341,#5a73eb


**plotting folium map**

In [48]:
map = folium.Map([51.5123,0.0910], zoom_start= 5)

for lat, lon, area, price in zip(df['latitude'],
                                 df['longitude'],
                                 df['neighbourhood_cleansed'],
                                 df['price']):
  
     folium.CircleMarker([lat, lon],
                         popup = area,
                         radius = np.log(price),
                         color='b',
                         fill=True,
                         fill_opacity=0.7,
                         fill_color=neighbourhood_colour_dict.get(area)).add_to(map)
map.save("london_map.html")